In [1]:
!pip install wandb

  Using cached wandb-0.12.5-py2.py3-none-any.whl (1.7 MB)
     |████████████████████████████████| 1.7 MB 1.3 MB/s eta 0:00:01
You should consider upgrading via the '/home/user/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable

from sklearn.decomposition import PCA

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

import wandb 

In [3]:
graph = pd.read_csv("graph_abakan_F_encoded.csv", index_col = 0)
X = pd.read_csv("abakan_full_routes_final_weather_L_NaN_filtered_FIXED.csv", index_col = 0).dropna().drop(["real_dist", "speed", "time", "length", "route_type","start_timestamp"], axis = 1).reset_index(drop=True)

In [4]:
X = X[(X["rebuildCount"] <= 1) & (X["RTA"] > 30) & (X["RTA"] < 3000)]
X = X.drop(["rebuildCount"], axis = 1)

In [6]:
X.reset_index(drop = True, inplace = True)

In [8]:
route_edges = np.array(X["edges"].map(lambda x: x.replace("'", "")))

In [9]:
wind_dir_classes = pd.get_dummies(X.wind_dir, prefix='wind_dir_class')
day_classes = pd.get_dummies(X.day_period, prefix='day_class')
Y = X["RTA"]
X = X.join(wind_dir_classes).join(day_classes).drop(["edges", "wind_dir", "day_period", "RTA"], axis = 1)

In [10]:
dense = X.iloc[:, 0:7].join(X[["temperature", "pressure", "wind_speed", "clouds", "snow"]])
sparse = X.iloc[:, 7::].drop(["temperature", "pressure", "wind_speed", "clouds", "snow"], axis =1)

In [11]:
pca = PCA(n_components=5)
sparse_compressed = pca.fit_transform(sparse)
deep_input = np.array(dense.join(pd.DataFrame(sparse_compressed))).astype("float32")

In [12]:
joined = dense.join(sparse)
features = list(joined.columns)
cross_product_features = []

for i in range(len(features)):
    for j in range(i+1, len(features)):
            cross_product_features.append([features[i], features[j]])

for i in range(len(cross_product_features)):
    joined[cross_product_features[i][0] + "_AND_" + cross_product_features[i][1]] = \
    joined[cross_product_features[i][0]]*joined[cross_product_features[i][1]]
    
wide_input = np.array(joined).astype("float32")

/home/user/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  # This is added back by InteractiveShellApp.init_path()


In [13]:
graph_features = graph.drop(["edge_id", "adjacent"], axis = 1)
embs = np.array(graph_features)

In [14]:
embs_dict = {}
for i in tqdm(range(len(route_edges))):
    arr = route_edges[i].split(",")
    route_embeddings = []
    for j in range(len(arr)):
        ind = graph[graph["edge_id"] == int(arr[j])].index
        if (len(ind) != 0):
            ind = ind[0]
        else:
#                print(str(i) + " " + str(j) + " " + arr[j] + " "+ str(graph[graph["edge_id"] == int(arr[j])].index[0]))
            print("index not found")
        route_embeddings.append(embs[ind])
    embs_dict[i] = route_embeddings

100%|██████████| 82202/82202 [12:06<00:00, 113.10it/s]


In [15]:
lens = set([len(embs_dict[i]) for i in range(len(embs_dict))])
batch_dict = {}
for i in tqdm(range(len(embs_dict))):
    if (len(embs_dict[i])) in batch_dict:
        batch_dict[len(embs_dict[i])].append([embs_dict[i], Y[i], i])
    else:
        batch_dict[len(embs_dict[i])] = []
        batch_dict[len(embs_dict[i])].append([embs_dict[i], Y[i], i])

100%|██████████| 82202/82202 [00:00<00:00, 150135.57it/s]


In [16]:
batch_list = list(batch_dict.values())

In [16]:
print(np.array(wide_input).shape, np.array(deep_input).shape)

(82202, 325) (82202, 17)


In [110]:
import wandb
wandb.init(project='ETA_second_stage', entity='eighonet')
wandb.run.name = "wdr_begin_lstm_full_deep_wide_final_filtered_SAMPLED"#"gs_3_ffd_3_128_test_MAEloss_lr_10^{-4}_10000"
wandb.run.save()



CondaEnvException: Unable to determine environment

Please re-run this command with one of the following options:

* Provide an environment name via --name or -n
* Re-run this command inside an activated conda environment.



wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


True

In [101]:
class WDR(nn.Module):
    def __init__(self, 
                 recurrent_input_size, wide_input_size, deep_input_size, 
                 lstm_hidden_size, lstm_num_layers, 
                 device):
        super(WDR, self).__init__()

        self.device = device
        
        # Recurrent part
        self.num_layers = lstm_num_layers
        self.hidden_size = lstm_hidden_size
        
        self.linear_preprocess_lstm = nn.Linear(recurrent_input_size, 256)
        self.lstm = nn.LSTM(input_size=256, hidden_size=lstm_hidden_size,
                    num_layers=lstm_num_layers, batch_first=True)
        
        # Deep part
        self.linear_1 = nn.Linear(deep_input_size, 256)
        self.linear_2 = nn.Linear(256, 256)
        self.linear_2_1 = nn.Linear(256, 256)
        
        # Wide part
        self.linear_3 = nn.Linear(wide_input_size, 256)
        
        # Final regression
        self.linear_4 = nn.Linear(512 + 256, 256)
        self.linear_5 = nn.Linear(256, 1)
        
        # Test layers
        self.linear_test = nn.Linear(256, 1)
        self.branch_test = nn.Linear(deep_input_size, 256)
        
        
    def forward(self, x, deep_input, wide_input, indices):
        def merge_deep_branch(h_out, indices):
            deep_output = Variable(torch.Tensor(deep_input[indices])).to(self.device)
            deep_output = F.relu(self.linear_1(deep_output))
            deep_output = F.relu(self.linear_2(deep_output))
            deep_output = F.relu(self.linear_2_1(deep_output))
            return torch.cat((h_out, deep_output), 1)
        
        def merge_wide_branch(final_input, indices):
            wide_output = Variable(torch.Tensor(wide_input[indices])).to(self.device)
            wide_output = F.relu(self.linear_3(wide_output))
            return torch.cat((final_input, wide_output), 1)
        
#        print(x.shape)
        preprocessed_lstm_data = F.relu(self.linear_preprocess_lstm(x))
        
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size)).to(self.device)
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size)).to(self.device)

        ula, (h_out, _) = self.lstm(preprocessed_lstm_data, (h_0, c_0))        
        h_out = h_out.view(-1, self.hidden_size)
        
        final_input = merge_deep_branch(h_out, indices)
        final_input = merge_wide_branch(final_input, indices)
        
        linear_out = F.relu(self.linear_4(final_input))
        linear_out = F.relu(self.linear_5(linear_out))
        return linear_out

In [106]:
num_epochs = 200
learning_rate = 0.000001

device = ""
gpu_ids = []
if torch.cuda.is_available():
    gpu_ids += [gpu_id for gpu_id in range(torch.cuda.device_count())]
    device = torch.device(f'cuda:{gpu_ids[0]}')
    torch.cuda.set_device(device)
else:
    device = torch.device('cpu')
print(device)
device = "cuda"

# Recurrent parameters
recurrent_input_size = np.array(batch_list[0][0][0]).shape[-1]
hidden_size = 256
num_layers = 1

# Deep parameters
deep_input_size = deep_input.shape[1]

# Wide parameters
wide_input_size = wide_input.shape[1]

wdr7 = WDR(recurrent_input_size, wide_input_size, deep_input_size,
          hidden_size, num_layers, 
          device)
wdr7 = wdr7.to(device)

criterion = torch.nn.L1Loss()
optimizer = torch.optim.Adam(wdr7.parameters(), lr=learning_rate)

cuda:0


In [107]:
ratio = 0.8
train_mask = {i for i in range(int(ratio*82200), 120000)} 
test_mask = {i for i in range(int(ratio*82200))}

In [113]:
wdr7.train()
for epoch in range(num_epochs):
    losses = []
    overall_loss, loss_test = 0, 0
    
    for i in tqdm(range(len(batch_list))):
        indices = np.array([batch_list[i][j][2] for j in range(len(batch_list[i]))])
        indices = np.array(list(set(indices) - train_mask))
        if len(indices) == 0:
            continue
        trainX = Variable(torch.Tensor(np.array([batch_list[i][j][0] for j in range(len(batch_list[i])) if batch_list[i][j][2] in indices])), requires_grad=True).to(device)
        trainY = Variable(torch.Tensor(np.array([batch_list[i][j][1] for j in range(len(batch_list[i])) if batch_list[i][j][2] in indices])), requires_grad=True).to(device)
        outputs = wdr7(x = trainX, deep_input = deep_input, wide_input = wide_input, indices = indices)
        optimizer.zero_grad()
#        print(torch.autograd.grad(outputs=outputs[0], inputs=wo, retain_graph=True)[0][0])
        # obtain the loss function
        loss = criterion(outputs, trainY.unsqueeze(-1))
        loss.backward(retain_graph=True)
        optimizer.step()
        losses.append(loss.item()/trainX.shape[0])
        overall_loss += loss.item()
    
    for i in tqdm(range(len(batch_list))):
        indices = np.array([batch_list[i][j][2] for j in range(len(batch_list[i]))])
        indices = np.array(list(set(indices) - test_mask))
        if len(indices) == 0:
            continue
        testX = Variable(torch.Tensor(np.array([batch_list[i][j][0] for j in range(len(batch_list[i])) if batch_list[i][j][2] in indices])), requires_grad=True).to(device)
        testY = Variable(torch.Tensor(np.array([batch_list[i][j][1] for j in range(len(batch_list[i])) if batch_list[i][j][2] in indices])), requires_grad=True).to(device)
        outputs = wdr7(x = testX, deep_input = deep_input, wide_input = wide_input, indices = indices)
        loss = criterion(outputs, testY.unsqueeze(-1))
        loss_test += loss.item()
        
    print(epoch, {"train_MAE": overall_loss/len(batch_list)}, {"test_MAE": loss_test/len(batch_list)})
    wandb.log({"train_MAE":  overall_loss/len(batch_list),
              "test_MAE": loss_test/len(batch_list)})

100%|██████████| 225/225 [00:13<00:00, 17.02it/s]


0 {'train_MAE': 489.5816154649523} {'test_MAE': 273.5765378994412}


100%|██████████| 225/225 [00:13<00:00, 16.42it/s] 


1 {'train_MAE': 328.7494549899631} {'test_MAE': 369.3745949130588}


100%|██████████| 225/225 [00:14<00:00, 15.19it/s] 


...

100%|██████████| 225/225 [00:13<00:00, 16.17it/s] 


197 {'train_MAE': 306.85791870117185} {'test_MAE': 199.36454530504014}


100%|██████████| 225/225 [00:13<00:00, 16.40it/s] 


198 {'train_MAE': 261.31643498738606} {'test_MAE': 197.58060406155056}


 76%|███████▌  | 170/225 [00:23<00:05, 10.48it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

